In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os
import sys

project_pth=os.path.join(os.getcwd(),'..','..')
sys.path.append(project_pth)

from utils.transformations import reusable

### **DimUser**


In [0]:
df=spark.read.format("parquet")\
  .load("abfss://bronze@project1storagespotify.dfs.core.windows.net/DimUser")

###**Autoloader**

In [0]:
df_user = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "parquet") \
    .option("cloudFiles.schemaLocation",
            "abfss://silver@project1storagespotify.dfs.core.windows.net/DimUser/schema") \
    .load("abfss://bronze@project1storagespotify.dfs.core.windows.net/DimUser")

In [0]:
df_user.isStreaming

In [0]:
df_user_obj = reusable()
df_user=df_user_obj.dropColumns(df_user,['_rescued_data',])



In [0]:
df_user.isStreaming


In [0]:
df_user.writeStream.format("delta")\
       .outputMode("append")\
       .option("checkpointLocation","abfss://silver@project1storagespotify.dfs.core.windows.net/DimUser/checkpoints")\
       .trigger(once=True)\
       .option("path","abfss://silver@project1storagespotify.dfs.core.windows.net/DimUser/data")\
       .toTable("spotify_cata.silver.DimUser")

###**DimArtist**

In [0]:
df_art = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "parquet") \
    .option("cloudFiles.schemaLocation",
            "abfss://silver@project1storagespotify.dfs.core.windows.net/DimArtist/schema") \
    .load("abfss://bronze@project1storagespotify.dfs.core.windows.net/DimArtist")

In [0]:
display(df_art)

In [0]:
df_art.isStreaming

In [0]:
df_art_obj=reusable()
df_art=df_art_obj.dropColumns(df_art,['_rescued_data'])
display(df_art)

In [0]:
df_art.isStreaming

In [0]:
df_art.writeStream.format("delta")\
       .outputMode("append")\
       .option("checkpointLocation","abfss://silver@project1storagespotify.dfs.core.windows.net/DimArtist/checkpoints")\
       .trigger(once=True)\
       .start("abfss://silver@project1storagespotify.dfs.core.windows.net/DimArtist/data")

In [0]:
df_art.writeStream.format("delta")\
       .outputMode("append")\
       .option("checkpointLocation","abfss://silver@project1storagespotify.dfs.core.windows.net/DimArtist/checkpoints")\
       .trigger(once=True)\
       .option("path","abfss://silver@project1storagespotify.dfs.core.windows.net/DimArtist/data")\
       .toTable("spotify_cata.silver.DimArtist")

###**DimTrack**

In [0]:
df_track = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "parquet") \
    .option("cloudFiles.schemaLocation",
            "abfss://silver@project1storagespotify.dfs.core.windows.net/DimTrack/schema") \
    .load("abfss://bronze@project1storagespotify.dfs.core.windows.net/DimTrack")

In [0]:
display(df_track)

In [0]:
df_track_obj = reusable()
df_track=df_track_obj.dropColumns(df_track,['_rescued_data'])
display(df_track)

In [0]:
df_track=df_track.withColumn("durationFlag",when(col('duration_sec')<150,"low")\
                                           .when(col('duration_sec')<300,"medium")\
                                           .otherwise("high"))


df_track=df_track.withColumn("track_name",regexp_replace(col('track_name'),'-',' '))

display(df_track)

In [0]:
df_track.writeStream.format("delta")\
       .outputMode("append")\
       .option("checkpointLocation","abfss://silver@project1storagespotify.dfs.core.windows.net/DimTrack/checkpoints")\
       .trigger(once=True)\
       .option("path","abfss://silver@project1storagespotify.dfs.core.windows.net/DimTrack/data")\
       .toTable("spotify_cata.silver.DimTrack")

###**DimDate**

In [0]:
df_date = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "parquet") \
    .option("cloudFiles.schemaLocation",
            "abfss://silver@project1storagespotify.dfs.core.windows.net/DimDate/schema") \
    .load("abfss://bronze@project1storagespotify.dfs.core.windows.net/DimDate")

In [0]:
df_date_obj = reusable()
df_date=df_date_obj.dropColumns(df_date,['_rescued_data'])


In [0]:
df_date.writeStream.format("delta")\
       .outputMode("append")\
       .option("checkpointLocation","abfss://silver@project1storagespotify.dfs.core.windows.net/DimDate/checkpoints")\
       .trigger(once=True)\
       .option("path","abfss://silver@project1storagespotify.dfs.core.windows.net/DimDate/data")\
       .toTable("spotify_cata.silver.DimDate")

###**FactStream**

In [0]:
df_fact = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "parquet") \
    .option("cloudFiles.schemaLocation",
            "abfss://silver@project1storagespotify.dfs.core.windows.net/FactStream/schema") \
    .load("abfss://bronze@project1storagespotify.dfs.core.windows.net/FactStream")

display(df_fact)

In [0]:
df_fact=reusable().dropColumns(df_fact,['_rescued_data'])

df_fact.writeStream.format("delta")\
       .outputMode("append")\
       .option("checkpointLocation","abfss://silver@project1storagespotify.dfs.core.windows.net/FactStream/checkpoints")\
       .trigger(once=True)\
       .option("path","abfss://silver@project1storagespotify.dfs.core.windows.net/FactStream/data")\
       .toTable("spotify_cata.silver.FactStream")